# Treeline
### (the ROOT equivalent of a *pipeline*)

In the big picutre, we don't have too much data and this doesn't need to be too automated. I've been focussed a lot on figuring out how to deal with the TTree format more efficiently, but that might not be the best way to go about it. Trying just to go w/ straightforward **slices** and iteration

In [ ]:
# imports, all in one place for ease of running
# apart from this, each sub-heading (e.g. Position, Calculating energy…) should be self-contained
# i.e. we import libraries and ROOT file here 

from ROOT import TFile, TProfile, TCanvas, TH1D, TH2D
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import math
from array import array

#For our rectangles
from matplotlib import cm
from matplotlib.patches import Rectangle

### Position
Using August and Charlie's normalized x-y positions, to plot for different detectors

In [ ]:
closest_alignment = 0 # <- 

#### And now using z-positions

### Exluding noisy pixels, and noisy events

### Excluding events with multiple tracks

### Calculating energy by magnet deflection

#### Finding $\theta$ by fitting a linear model

#### And calcuating angle using, as Cristóvão notes,
$$ \sin\theta = 0.3\frac{L\cdot B}{p} $$

### Deposited calorimter energy
(with our favourite rectangular visualizations!)

---
### And condensing it to do in parallel to compare different runs

In [6]:
# Above, I've noted with `# <-` places where we can reference our spreadsheet to import needed data
# Let's put that to use!